# Proxy Pattern

## Intent
Provide a surrogate or placeholder for another object to control access to it.

## Problem
You need to control access to an object because:
- Object creation is expensive (lazy initialization)
- Need access control (authentication/authorization)
- Object is in remote location (network communication)
- Want to add logging, caching, or monitoring
- Need reference counting or cleanup

**Real-world analogy**: Credit card is a proxy for your bank account

## When to Use
✅ **Use when:**
- Need lazy initialization (virtual proxy)
- Need access control (protection proxy)
- Object is remote (remote proxy)
- Want to cache results (cache proxy)
- Need logging or monitoring (smart reference)

❌ **Avoid when:**
- Direct access is simple and sufficient
- Overhead of proxy is not justified
- No need for access control or lazy loading

## Pattern Structure
```
┌────────┐         ┌─────────┐
│ Client │────────►│ Subject │
└────────┘         │Interface│
                   └─────────┘
                        ▲
              ┌─────────┴──────────┐
         ┌────┴────┐         ┌─────┴─────┐
         │  Proxy  │────────►│ RealSubj  │
         ├─────────┤         │           │
         │realSubj │         └───────────┘
         │request()│
         └─────────┘
```

## Example 1: Image Loading (Without Proxy)

**Problem**: Large images loaded immediately, even if not displayed

In [ ]:
import time

# WITHOUT Proxy - Expensive loading happens immediately
class Image:
    def __init__(self, filename: str):
        self.filename = filename
        self._load_from_disk()  # Expensive operation!
    
    def _load_from_disk(self):
        print(f"💾 Loading {self.filename} from disk...")
        time.sleep(1)  # Simulate slow loading
        print(f"✅ Loaded {self.filename}")
    
    def display(self):
        print(f"🖼️  Displaying {self.filename}")

# Problem: All images load immediately!
print("Creating image objects...")
img1 = Image("photo1.jpg")  # Loads immediately
img2 = Image("photo2.jpg")  # Loads immediately
img3 = Image("photo3.jpg")  # Loads immediately

print("\nNow displaying only one image:")
img1.display()  # Other images loaded but never used!

## Implementation: Virtual Proxy (Lazy Loading)

In [ ]:
from abc import ABC, abstractmethod
import time

# Subject interface
class ImageInterface(ABC):
    """Common interface for images."""
    
    @abstractmethod
    def display(self) -> None:
        pass


# Real subject (expensive to create)
class RealImage(ImageInterface):
    """Real image that loads from disk."""
    
    def __init__(self, filename: str):
        self.filename = filename
        self._load_from_disk()
    
    def _load_from_disk(self) -> None:
        print(f"  💾 Loading {self.filename} from disk...")
        time.sleep(0.5)  # Simulate slow loading
        print(f"  ✅ Loaded {self.filename}")
    
    def display(self) -> None:
        print(f"  🖼️  Displaying {self.filename}")


# Proxy (controls access)
class ImageProxy(ImageInterface):
    """Proxy that delays loading until needed."""
    
    def __init__(self, filename: str):
        self.filename = filename
        self._real_image: RealImage = None  # Not loaded yet!
    
    def display(self) -> None:
        # Lazy loading: create real image only when needed
        if self._real_image is None:
            print(f"[Proxy] First access to {self.filename}")
            self._real_image = RealImage(self.filename)
        else:
            print(f"[Proxy] Using cached {self.filename}")
        
        self._real_image.display()


# Demo
print("=== Virtual Proxy (Lazy Loading) ===")

print("\n1. Creating proxy objects (instant):")
img1 = ImageProxy("photo1.jpg")  # Instant!
img2 = ImageProxy("photo2.jpg")  # Instant!
img3 = ImageProxy("photo3.jpg")  # Instant!
print("   All proxies created (no loading yet)\n")

print("2. Displaying image 1 (loads on first access):")
img1.display()

print("\n3. Displaying image 1 again (uses cached):")
img1.display()

print("\n4. Displaying image 2 (loads on first access):")
img2.display()

print("\n5. Image 3 never used - never loaded!")

print("\n✅ Lazy loading: images only load when needed!")

## Real-World Example: Protection Proxy (Access Control)

In [ ]:
from typing import Optional

# Subject interface
class BankAccount(ABC):
    """Bank account interface."""
    
    @abstractmethod
    def deposit(self, amount: float) -> None:
        pass
    
    @abstractmethod
    def withdraw(self, amount: float) -> None:
        pass
    
    @abstractmethod
    def get_balance(self) -> float:
        pass


# Real subject
class RealBankAccount(BankAccount):
    """Real bank account."""
    
    def __init__(self, account_number: str, initial_balance: float = 0):
        self.account_number = account_number
        self._balance = initial_balance
    
    def deposit(self, amount: float) -> None:
        self._balance += amount
        print(f"  💵 Deposited ${amount:.2f}. Balance: ${self._balance:.2f}")
    
    def withdraw(self, amount: float) -> None:
        if amount <= self._balance:
            self._balance -= amount
            print(f"  💸 Withdrew ${amount:.2f}. Balance: ${self._balance:.2f}")
        else:
            print(f"  ❌ Insufficient funds")
    
    def get_balance(self) -> float:
        return self._balance


# Protection proxy
class BankAccountProxy(BankAccount):
    """Proxy with access control."""
    
    def __init__(self, account_number: str, password: str, initial_balance: float = 0):
        self._real_account = RealBankAccount(account_number, initial_balance)
        self._password = password
        self._authenticated = False
    
    def authenticate(self, password: str) -> bool:
        """Authenticate user."""
        if password == self._password:
            self._authenticated = True
            print("🔓 Authentication successful")
            return True
        else:
            self._authenticated = False
            print("🔒 Authentication failed")
            return False
    
    def _check_access(self) -> bool:
        """Check if user is authenticated."""
        if not self._authenticated:
            print("  ❌ Access denied. Please authenticate first.")
            return False
        return True
    
    def deposit(self, amount: float) -> None:
        if self._check_access():
            self._real_account.deposit(amount)
    
    def withdraw(self, amount: float) -> None:
        if self._check_access():
            self._real_account.withdraw(amount)
    
    def get_balance(self) -> float:
        if self._check_access():
            return self._real_account.get_balance()
        return 0.0


# Demo
print("\n=== Protection Proxy (Access Control) ===")

account = BankAccountProxy("ACC123", "secret123", 1000.0)

print("\n1. Try to access without authentication:")
account.get_balance()
account.deposit(100)

print("\n2. Authenticate with wrong password:")
account.authenticate("wrong")

print("\n3. Authenticate with correct password:")
account.authenticate("secret123")

print("\n4. Now access is granted:")
balance = account.get_balance()
print(f"  💰 Current balance: ${balance:.2f}")
account.deposit(500)
account.withdraw(200)

print("\n✅ Proxy controls access to real object!")

## Real-World Example: Cache Proxy

In [ ]:
from typing import Dict
import time

# Subject interface
class Database(ABC):
    """Database interface."""
    
    @abstractmethod
    def query(self, sql: str) -> list:
        pass


# Real subject
class RealDatabase(Database):
    """Real database with slow queries."""
    
    def query(self, sql: str) -> list:
        print(f"  🗄️  Executing query: {sql}")
        time.sleep(0.5)  # Simulate slow query
        
        # Mock results
        if "users" in sql:
            result = [{"id": 1, "name": "Alice"}, {"id": 2, "name": "Bob"}]
        elif "products" in sql:
            result = [{"id": 1, "name": "Laptop"}, {"id": 2, "name": "Mouse"}]
        else:
            result = []
        
        print(f"  ✅ Query returned {len(result)} rows")
        return result


# Cache proxy
class CachingDatabaseProxy(Database):
    """Proxy that caches query results."""
    
    def __init__(self):
        self._real_db = RealDatabase()
        self._cache: Dict[str, list] = {}
    
    def query(self, sql: str) -> list:
        if sql in self._cache:
            print(f"  ⚡ Cache hit! Returning cached result")
            return self._cache[sql]
        
        print(f"  📂 Cache miss. Querying database...")
        result = self._real_db.query(sql)
        self._cache[sql] = result
        return result
    
    def clear_cache(self) -> None:
        """Clear the cache."""
        self._cache.clear()
        print("  🗑️  Cache cleared")


# Demo
print("\n=== Cache Proxy ===")

db = CachingDatabaseProxy()

print("\n1. First query (slow - cache miss):")
start = time.time()
result1 = db.query("SELECT * FROM users")
print(f"   Time: {time.time() - start:.2f}s")
print(f"   Result: {result1}")

print("\n2. Same query again (fast - cache hit):")
start = time.time()
result2 = db.query("SELECT * FROM users")
print(f"   Time: {time.time() - start:.2f}s")
print(f"   Result: {result2}")

print("\n3. Different query (slow - cache miss):")
start = time.time()
result3 = db.query("SELECT * FROM products")
print(f"   Time: {time.time() - start:.2f}s")
print(f"   Result: {result3}")

print("\n4. Repeat product query (fast - cache hit):")
start = time.time()
result4 = db.query("SELECT * FROM products")
print(f"   Time: {time.time() - start:.2f}s")

print("\n✅ Proxy caches expensive operations!")

## Real-World Example: Logging Proxy (Smart Reference)

In [ ]:
from datetime import datetime

# Subject interface
class FileSystem(ABC):
    """File system interface."""
    
    @abstractmethod
    def read_file(self, filename: str) -> str:
        pass
    
    @abstractmethod
    def write_file(self, filename: str, content: str) -> None:
        pass
    
    @abstractmethod
    def delete_file(self, filename: str) -> None:
        pass


# Real subject
class RealFileSystem(FileSystem):
    """Real file system."""
    
    def __init__(self):
        self._files: Dict[str, str] = {}
    
    def read_file(self, filename: str) -> str:
        return self._files.get(filename, "")
    
    def write_file(self, filename: str, content: str) -> None:
        self._files[filename] = content
    
    def delete_file(self, filename: str) -> None:
        if filename in self._files:
            del self._files[filename]


# Logging proxy
class LoggingFileSystemProxy(FileSystem):
    """Proxy that logs all operations."""
    
    def __init__(self, user: str):
        self._real_fs = RealFileSystem()
        self._user = user
    
    def _log(self, operation: str, filename: str, details: str = "") -> None:
        """Log the operation."""
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        print(f"📝 [{timestamp}] User: {self._user} | {operation}: {filename} {details}")
    
    def read_file(self, filename: str) -> str:
        self._log("READ", filename)
        content = self._real_fs.read_file(filename)
        self._log("READ", filename, f"({len(content)} bytes)")
        return content
    
    def write_file(self, filename: str, content: str) -> None:
        self._log("WRITE", filename, f"({len(content)} bytes)")
        self._real_fs.write_file(filename, content)
        self._log("WRITE", filename, "[COMPLETED]")
    
    def delete_file(self, filename: str) -> None:
        self._log("DELETE", filename)
        self._real_fs.delete_file(filename)
        self._log("DELETE", filename, "[COMPLETED]")


# Demo
print("\n=== Logging Proxy (Smart Reference) ===")

fs = LoggingFileSystemProxy(user="alice")

print("\n1. Write file:")
fs.write_file("document.txt", "Hello, World!")

print("\n2. Read file:")
content = fs.read_file("document.txt")
print(f"   Content: {content}")

print("\n3. Write another file:")
fs.write_file("data.json", '{"key": "value"}')

print("\n4. Delete file:")
fs.delete_file("document.txt")

print("\n✅ Proxy logs all operations!")

## Types of Proxies

### 1. Virtual Proxy
Delays creation of expensive objects until needed.
```python
class ImageProxy:
    def display(self):
        if not self._real_image:
            self._real_image = RealImage()  # Lazy loading
```

### 2. Protection Proxy
Controls access based on permissions.
```python
class BankAccountProxy:
    def withdraw(self, amount):
        if self._is_authenticated:
            self._real_account.withdraw(amount)
```

### 3. Remote Proxy
Represents object in different address space (e.g., RPC, REST API).
```python
class RemoteServiceProxy:
    def call_method(self):
        response = requests.post(url, data)
```

### 4. Cache Proxy
Caches results of expensive operations.
```python
class CacheProxy:
    def query(self, sql):
        if sql in cache:
            return cache[sql]
```

### 5. Smart Reference
Adds additional behavior (logging, reference counting).
```python
class LoggingProxy:
    def method(self):
        log("method called")
        return self._real_obj.method()
```

## Proxy vs Decorator vs Adapter

**Proxy**:
- Same interface as real object
- Controls access to real object
- May create real object lazily

**Decorator**:
- Same interface
- Adds behavior
- Real object always exists

**Adapter**:
- Different interface
- Converts interface
- For incompatible interfaces

In [ ]:
# Comparison

# Proxy - Controls access, same interface
class ServiceProxy:
    def __init__(self):
        self._service = None  # Created lazily
    
    def request(self):
        if not self._service:
            self._service = RealService()  # Lazy creation
        return self._service.request()


# Decorator - Adds behavior, same interface
class EncryptionDecorator:
    def __init__(self, service):
        self._service = service  # Service already exists
    
    def request(self):
        result = self._service.request()
        return self._encrypt(result)  # Adds encryption


# Adapter - Converts interface
class PayPalAdapter:
    def __init__(self, paypal):
        self._paypal = paypal
    
    def pay(self, amount):  # Different interface
        self._paypal.send_payment(amount)  # Adapts to PayPal's interface


print("Proxy: Controls access (lazy loading, auth, caching)")
print("Decorator: Adds behavior (encryption, logging)")
print("Adapter: Converts interface (makes incompatible work together)")

## Advantages & Disadvantages

### ✅ Advantages
1. **Lazy initialization**: Create expensive objects only when needed
2. **Access control**: Add security without modifying real object
3. **Additional functionality**: Logging, caching, monitoring
4. **Open/Closed Principle**: Add proxies without modifying real object
5. **Transparent**: Client doesn't know if using proxy or real object

### ❌ Disadvantages
1. **Complexity**: Additional layer of indirection
2. **Performance**: Extra overhead (unless caching)
3. **Response delay**: Lazy loading causes first-access delay

## Common Use Cases

1. **Lazy loading**: Images, large documents, database connections
2. **Access control**: Authentication, authorization, permissions
3. **Remote objects**: RPC, REST APIs, microservices
4. **Caching**: Database queries, API calls, computations
5. **Logging/monitoring**: Track access, usage statistics
6. **Reference counting**: Smart pointers, resource management
7. **Network optimization**: Buffering, batching requests

## Related Patterns

- **Adapter**: Changes interface (Proxy keeps same interface)
- **Decorator**: Adds behavior (Proxy controls access)
- **Facade**: Simplifies interface (Proxy controls access)
- **Flyweight**: Often uses proxy for shared objects

## Best Practices

1. **Keep interface identical**: Proxy should match real object interface
2. **Thread safety**: Consider synchronization for lazy loading
3. **Clear purpose**: Choose appropriate proxy type
4. **Document behavior**: Explain what proxy adds/controls
5. **Consider performance**: Proxy adds overhead
6. **Handle errors**: Proxy should handle real object failures gracefully

## Summary

Proxy pattern enables:
- Controlled access to objects
- Lazy initialization
- Additional functionality
- Transparent substitution

Perfect for: Lazy loading, access control, remote objects, caching, logging.

**Key Insight**: Provide a surrogate that controls access to the real object, adding functionality without changing the interface!